부분적인 코딩은 다함
0. 폴더 변수명, 폴더 생성자 이름, 폴더 참조명 변수 정하기
1. dicom파일들 읽어온 후 array로 받기
2. rt structure file을 nii데이터로 옮긴후 nii data를 통합해서 받기(rtstructure2nii)
3. nii data를 array로 받아 저장하기
4. 그러면 train_x에는 다이콤에서 받은 어레이가 3차원으로 들어가고
5. train_y는 변형된 nii data로 들어가게 된다.



In [3]:
import numpy as np
import glob
import os
import nibabel as nib
from skimage import measure
import numpy
import pandas as pd
import pydicom
import os
import numpy
import numpy as np
import glob
import nibabel as nib
from skimage import measure 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import time
import glob
import pydicom
import numpy as np
from vtkplotter import Volume
import sys, os


In [4]:
cd C:\Users\razer\Desktop\new_npy

C:\Users\razer\Desktop\new_npy


In [7]:
CT_dicom_RT_Structure_folder = r'ct+rt folder' #폴더 내에 같은 환자에 원래 ct영상, rt영상 ,ct영상, rt영상 ,ct영상, rt영상  이런식으로 구성
CT_dicom_RT_Structure_name = os.listdir(CT_dicom_RT_Structure_folder) # 전체 폴더 리스트 불러오기
CT_dicom = glob.glob('?????????*CT*') # 전체 폴더중 ct라는 글자가 들어가면 ct dicom에 넣어라
RT_Structure = [] # rt structrure의 폴더는 빈 리스트로 두었다가 전체폴더리스트에서 ct 리스트를 뺄것임
for i in CT_dicom_RT_Structure_name[0:]: # 빼는 과정
    if i not in CT_dicom:
        RT_Structure.append(i)
        
        
lstFilesDCM = []  # create an empty list


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'ct+rt folder'

In [6]:
len(CT_dicom),len(RT_Structure),len(CT_dicom_RT_Structure_name)

(0, 113, 113)

In [5]:
CT_dicom

['P001_HN1_CT_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n150__00000',
 'P002_HN2_CT_2018-09-20_145605_._Neck.3.0_n157__00000',
 'P003_HN3_CT_2018-06-11_120112_._Neck.3.0_n163__00000',
 'P004_HN4_CT_2018-05-30_172827_._Neck.3.0_n149__00000',
 'P005_HN5_CT_2018-08-20_111501_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n162__00000',
 'P006_HN6_CT_2018-06-12_115513_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n163__00000',
 'P007_HN7_CT_2018-08-08_145352_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n165__00000',
 'P008_HN8_CT_2018-02-09_165554_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n149__00001',
 'P009_HN9_CT_2018-09-13_095458_._Neck.3.0_n161__00000',
 'P010_HN10_CT_2018-11-06_104704_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n158__00001',
 'P011_HN11_CT_2018-01-04_165105_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n173__00001',
 'P012_HN12_CT_2018-11-12_170733_._Neck.3.0_n162__00001',
 'P013_HN13_CT_2018-03-09_084340_._Neck.

In [6]:
RT_Structure

['P001_HN1_RTst_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_re_n1__00000',
 'P002_HN2_RTst_2018-09-20_145605_._RE_n1__00000',
 'P003_HN3_RTst_2018-06-11_120112_._OAR3RE_n1__00000',
 'P004_HN4_RTst_2018-05-30_172827_._OAR4RE_n1__00000',
 'P005_HN5_RTst_2018-08-20_111501_RT^01.RT.Head.Neck.(Adult)_OAR3RD_n1__00000',
 'P006_HN6_RTst_2018-06-12_115513_RT^01.RT.Head.Neck.(Adult)_OAR6RE_n1__00000',
 'P007_HN7_RTst_2018-08-08_145352_RT^01.RT.Head.Neck.(Adult)_OAR7RE_n1__00000',
 'P008_HN8_RTst_2018-02-09_165554_RT^01.RT.Head.Neck.(Adult)_RE_n1__00001',
 'P009_HN9_RTst_2018-09-13_095458_._RE_n1__00000',
 'P010_HN10_RTst_2018-11-06_104704_RT^01.RT.Head.Neck.(Adult)_OAR10RE_n1__00000',
 'P011_HN11_RTst_2018-01-04_165105_RT^01.RT.Head.Neck.(Adult)_OAR11RE_n1__00000',
 'P012_HN12_RTst_2018-11-12_170733_._OAR12_n1__00000',
 'P013_HN13_RTst_2018-03-09_084340_._OAR13_n1__00000',
 'P014_HN14_RTst_2018-09-12_145420_RT^01.RT.Head.Neck.(Adult)_OAR14RE_n1__00000',
 'P015_HN15_RTst_2018-06-01_115954_._RE_

In [104]:


total_dict_files_dcm = {} # 딕트형식에 key값이 환자폴더 이름이 들어가고, value값에 어레이가 들어갈 것임
lstFilesDCM = []  # create an empty list xy의 2차원 다이콤 ct 데이터를 쌓을 리스트
for l in CT_dicom[0:1]: #  폴더 1개만 불러와서
    PathDicom =CT_dicom_RT_Structure_folder + l # path + i 만큼 pathdicom의 경로가 만들어져서 폴더를 읽어올거임
    for dirName, subdirList, fileList in os.walk(PathDicom): # 폴더안에들어가서 filename이 .dcm으로 끝나면 lstfilesDCM에 추가하는 형식
        for filename in fileList:
            if ".dcm" in filename.lower():  # check whether the file's DICOM
                lstFilesDCM.append(os.path.join(dirName,filename))
                
                # Get ref file
                RefDs = pydicom.read_file(lstFilesDCM[0]) # 이부분은 따옴  https://www.programcreek.com/python/example/97517/dicom.read_file

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
                ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))

# Load spacing values (in mm)
                ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))
    
    # The array is sized based on 'ConstPixelDims'
    ArrayDicom = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
    for filenameDCM in tqdm(lstFilesDCM):
        # read the file
        ds = pydicom.read_file(filenameDCM)
        # store the raw image data
        ArrayDicom[:, :, lstFilesDCM.index(filenameDCM)] = ds.pixel_array
       
        total_dict_files_dcm[l] = ArrayDicom
        

    











  0%|                                                                                          | 0/150 [00:00<?, ?it/s]









  2%|█▋                                                                                | 3/150 [00:00<00:07, 20.52it/s]









  3%|██▋                                                                               | 5/150 [00:00<00:07, 18.99it/s]









  5%|███▊                                                                              | 7/150 [00:00<00:07, 18.05it/s]









  6%|████▉                                                                             | 9/150 [00:00<00:08, 17.49it/s]









  7%|█████▉                                                                           | 11/150 [00:00<00:08, 17.03it/s]









  9%|███████                                                                          | 13/150 [00:00<00:08, 16.68it/s]









 10%|████████                                                                         |

In [105]:
Volume(total_dict_files_dcm['P001_HN1_CT_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n150__00000']).show(bg="black")
# 3d visualization code

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[262.87863808571745…

In [7]:
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs

In [8]:
niidata = r'\\10.28.51.43\mpbel\Meeting Materials\Kanghoun\HN_data\new_npy/'
lstFilesDCM = []
for q in RT_Structure[0:]:
    PathDicom =CT_dicom_RT_Structure_folder + q
    for dirName, subdirList, fileList in os.walk(PathDicom):
        for filename in fileList:
            if ".dcm" in filename.lower():  # check whether the file's DICOM
                lstFilesDCM.append(os.path.join(dirName,filename))
                


In [9]:
len(lstFilesDCM)

113

In [10]:
len(RT_Structure)

113

In [12]:
for i in RT_Structure: # 이건 각 rt structure파일들이 담긴 폴더와 같은 폴더를 만들어서 nii.gz파일을 만들어 넣어둘거임
    os.mkdir(niidata +'/'+i) 
#     한번만 하면 됨
import itertools

In [11]:
from tqdm import tqdm # 진행상황 확인

In [12]:
import itertools # 파일들을 짤 때 for문을 세번돌리면 기하급수적으로 느려지므로 합쳐서 돌릴거임 서로 len은 113개로 같음
save_list = []
save_list =  list(zip(CT_dicom,RT_Structure,range(113)))


In [13]:
save_list

[('P001_HN1_CT_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n150__00000',
  'P001_HN1_RTst_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_re_n1__00000',
  0),
 ('P002_HN2_CT_2018-09-20_145605_._Neck.3.0_n157__00000',
  'P002_HN2_RTst_2018-09-20_145605_._RE_n1__00000',
  1),
 ('P003_HN3_CT_2018-06-11_120112_._Neck.3.0_n163__00000',
  'P003_HN3_RTst_2018-06-11_120112_._OAR3RE_n1__00000',
  2),
 ('P004_HN4_CT_2018-05-30_172827_._Neck.3.0_n149__00000',
  'P004_HN4_RTst_2018-05-30_172827_._OAR4RE_n1__00000',
  3),
 ('P005_HN5_CT_2018-08-20_111501_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n162__00000',
  'P005_HN5_RTst_2018-08-20_111501_RT^01.RT.Head.Neck.(Adult)_OAR3RD_n1__00000',
  4),
 ('P006_HN6_CT_2018-06-12_115513_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n163__00000',
  'P006_HN6_RTst_2018-06-12_115513_RT^01.RT.Head.Neck.(Adult)_OAR6RE_n1__00000',
  5),
 ('P007_HN7_CT_2018-08-08_145352_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n165__00000',

In [20]:
#113개의 폴더를 만들어 ct원래영상과 rtstructure의 dicom파일과 원영상, rt structure변환 영상을 만들어진 폴더 내로 넣을거임
#sequence 오류가 생기면 나도 잘 모르겠음

for i,w,k in tqdm(save_list[0:]):
    PathDicom2 = CT_dicom_RT_Structure_folder + i
    PathDicom1 = niidata + w
            
    dcmrtstruct2nii(str(lstFilesDCM[k]),str(PathDicom2),PathDicom1)
    

100%|██████████████████████████████████████████████████████████████████████████████| 113/113 [1:26:39<00:00, 46.01s/it]
